# Defect Detection in Additive Manufacturing Microscopic Images

In [1]:
from utils import *
from model import *

2024-04-01 09:49:39.523777: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-01 09:49:39.548905: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-01 09:49:40.022458: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


#### Uncomment and Run the below code if cvat is not cloned.

In [3]:
# !git clone https://github.com/Hitesh1309/cvat.git

### After cloning the cvat repository, make sure you complete the installation.

#### Uncomment and Run the below code if installation is not Done.

In [7]:
installation_guide()

### Run the below code to bring up cvat.

In [8]:
run_docker_compose_and_return('./cvat')

All containers are up and running.


 Container cvat_opa  Running
 Container cvat_redis_ondisk  Running
 Container cvat_redis_inmem  Running
 Container nuclio  Running
 Container cvat_clickhouse  Running
 Container cvat_db  Running
 Container traefik  Running
 Container cvat_worker_export  Running
 Container cvat_worker_import  Running
 Container cvat_worker_analytics_reports  Running
 Container cvat_grafana  Running
 Container cvat_utils  Running
 Container cvat_vector  Running
 Container cvat_worker_webhooks  Running
 Container cvat_worker_annotation  Running
 Container cvat_worker_quality_reports  Running
 Container cvat_server  Running
 Container cvat_ui  Running


In [9]:
open_cvat()

In [10]:
open_nuclio()

#### Make sure you have the training images which have the ground truth in the folder named *Training Images*, which has two subfolders - Images and Masks. 

In [2]:
# change the path to the folder which has the images
folder_path = './Training Images'

count_files_in_folder(folder_path)

Number of files in the folder './Training Images/Masks': 2
Number of files in the folder './Training Images/Images': 2


### Create the job in CVAT. 
Refer - https://opencv.github.io/cvat/docs/api_sdk/cli/

In [ ]:
# write code for creating a job from the command line

### Training the Model.

In [3]:
# Load the data for training the model.
images, masks = load_images_and_masks(folder_path)
print(images.shape, masks.shape)

(2, 480, 640, 1) (2, 480, 640, 1)


In [9]:
# Instantiate and compile the U-Net model
model = unet(input_size=(480, 640, 1))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model 
history = model.fit(images, masks, epochs=10, batch_size=1)

Epoch 1/10


2024-04-01 09:54:38.786152: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-01 09:54:38.818374: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


2/2 ━━━━━━━━━━━━━━━━━━━━ 191s 95s/step - accuracy: 0.9088 - loss: 0.5499
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 192s 96s/step - accuracy: 0.9797 - loss: 0.4685
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 193s 96s/step - accuracy: 0.9845 - loss: 0.3852
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 193s 97s/step - accuracy: 0.9776 - loss: 0.3032
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 193s 97s/step - accuracy: 0.9783 - loss: 0.2308
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 194s 97s/step - accuracy: 0.9827 - loss: 0.1783
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 194s 97s/step - accuracy: 0.9843 - loss: 0.1472
Epoch 8/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 194s 97s/step - accuracy: 0.9856 - loss: 0.1219
Epoch 9/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 192s 96s/step - accuracy: 0.9891 - loss: 0.1077
Epoch 10/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 191s 95s/step - accuracy: 0.9883 - loss: 0.0935


#### Saving the model

In [11]:
# save the model as model_new.keras
model.save("model_new.keras")

### Upload the model in the Google Drive following the below link. 
(Find an alternate way)

In [4]:
from IPython.display import display, HTML
display(HTML('<div id="target_cell"></div>'))

In [5]:
open_google_drive_folder()

### Deploy the model into CVAT

In [6]:
!./cvat/serverless/deploy_cpu.sh ./cvat/serverless/unet-coco-20th-march/

[+] Building 0.0s (0/1)                                          docker:default
[+] Building 0.2s (1/2)                                          docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 418B                                       0.0s
 => [internal] load metadata for docker.io/openvino/ubuntu20_runtime:2022  0.2s
[+] Building 0.3s (1/2)                                          docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 418B                                       0.0s
 => [internal] load metadata for docker.io/openvino/ubuntu20_runtime:2022  0.3s
[+] Building 0.5s (1/2)                                          docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 418B                                       0.0s
 => [internal] load metadata for docker.

24.04.01 09:50:16.974            nuctl.platform (I) Building docker image {"image": "cvat.unet-coco:latest"}
24.04.01 09:50:19.244            nuctl.platform (I) Pushing docker image into registry {"image": "cvat.unet-coco:latest", "registry": ""}
24.04.01 09:50:19.244            nuctl.platform (I) Docker image was successfully built and pushed into docker registry {"image": "cvat.unet-coco:latest"}
24.04.01 09:50:19.244                     nuctl (I) Build complete {"image": "cvat.unet-coco:latest"}
24.04.01 09:50:19.249                     nuctl (I) Cleaning up before deployment {"functionName": "unet-coco"}
24.04.01 09:50:19.266                     nuctl (I) Function already exists, deleting function containers {"functionName": "unet-coco"}
24.04.01 09:50:20.283            nuctl.platform (I) Waiting for function to be ready {"timeout": 120}
24.04.01 09:50:21.968                     nuctl (I) Function deploy complete {"functionName": "unet-coco", "httpPort": 12345, "internalInvocationU

### Open the Job in CVAT and Select AI Tools --> Detectors --> U-Net. 
#### Select "defect" in both the dropdowns and check the "Convert masks to polygons" option.
#### Press "Annotate" and then make the necessary changes manually.

### After the manual correction, press the "Menu" option, Select "Export job dataset", in the Export format, Scroll down to "Segmentation mask 1.1", type in the name for the zip file and click "OK".

In [7]:
new_folder_path = './Annotated Images'

count_files_in_folder(new_folder_path)

Number of files in the folder './Annotated Images/Masks': 1
Number of files in the folder './Annotated Images/Images': 1


In [12]:
new_model = tf.keras.models.load_model('model_new.keras')

### Loading the new data for training the model.

In [4]:
# Load the new data for training the model.
images, masks = load_images_and_masks(new_folder_path)
print(images.shape, masks.shape)

(1, 480, 640, 1) (1, 480, 640, 1)


### Training the model on new data.

In [7]:
# Train the model 
history = new_model.fit(images, masks, epochs=10, batch_size=1)

Epoch 1/10
1/1 [==============================] - 20s 20s/step - loss: 0.1488 - accuracy: 0.9720
Epoch 2/10
1/1 [==============================] - 21s 21s/step - loss: 0.1382 - accuracy: 0.9727
Epoch 3/10
1/1 [==============================] - 33s 33s/step - loss: 0.1305 - accuracy: 0.9736
Epoch 4/10
1/1 [==============================] - 44s 44s/step - loss: 0.1261 - accuracy: 0.9741
Epoch 5/10
1/1 [==============================] - 51s 51s/step - loss: 0.1207 - accuracy: 0.9743
Epoch 6/10
1/1 [==============================] - 36s 36s/step - loss: 0.1149 - accuracy: 0.9744
Epoch 7/10
1/1 [==============================] - 34s 34s/step - loss: 0.1107 - accuracy: 0.9746
Epoch 8/10
1/1 [==============================] - 36s 36s/step - loss: 0.1081 - accuracy: 0.9747
Epoch 9/10
1/1 [==============================] - 30s 30s/step - loss: 0.1058 - accuracy: 0.9748
Epoch 10/10
1/1 [==============================] - 32s 32s/step - loss: 0.1024 - accuracy: 0.9750


In [8]:
# save the model as model_new.keras
new_model.save("model_new.keras")

In [14]:
from IPython.display import display, Javascript

display(Javascript('document.getElementById("target_cell").scrollIntoView()'))

<IPython.core.display.Javascript object>